# Chapter 14 반복형, 반복자, 제너레이터

## 14.1 Sentence 버전 #1: 단어 시퀀스

In [2]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
    
    def __getitem__(self, index):
        return self.words[index]
    
    def __len__(self):
        return len(self.words)
    
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    
s = Sentence('"The time has come," the Walrus said,')
s

Sentence('"The time ha... Walrus said,')

In [3]:
for word in s:
    print(word)

The
time
has
come
the
Walrus
said


In [6]:
s[0], s[5], s[-1]

('The', 'Walrus', 'said')

In [9]:
class Foo:
    def __iter__():
        pass
    
from collections import abc
issubclass(Foo, abc.Iterable)

True

In [11]:
f = Foo()
isinstance(f, abc.Iterable)

True

In [13]:
issubclass(Sentence, abc.Iterable)

False

## 14.2 반복형과 반복자

In [14]:
s = 'ABC'
for char in s:
    print(char)

A
B
C


In [15]:
s = 'ABC'
it = iter(s)
while True:
    try:
        print(next(it))
    except StopIteration:
        del it
        break

A
B
C


In [23]:
s3 = Sentence('Pig and Pepper')
it = iter(s3)
it

In [24]:
next(it), next(it), next(it)

('Pig', 'and', 'Pepper')

In [25]:
next(it)

StopIteration: 

In [26]:
list(it)

[]

In [27]:
list(iter(s3))

['Pig', 'and', 'Pepper']

## 14.3 Sentence 버전 #2: 고전적인 반복자

In [30]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __len__(self):
        return len(self.words)
    
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    
    def __iter__(self):
        return SentenceIterator(self.words)
    
class SentenceIterator:
    def __init__(self, words):
        self.words = words
        self.index = 0
        
    def __next__(self):
        try:
            word = self.words[self.index]
        except IndexError:
            raise StopIteration()
        self.index += 1
        return word
    
    def __iter__(self):
        return self
    
s = Sentence('"The time has come," the Walrus said,')
s

Sentence('"The time ha... Walrus said,')

In [32]:
issubclass(SentenceIterator, abc.Iterable)

True

In [31]:
for word in s:
    print(word)

The
time
has
come
the
Walrus
said


## 14.4 Sentence 버전 #3: 제네레이터 함수

In [1]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
        
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    
    def __iter__(self):
        for word in self.words:
            yield word
        
s = Sentence('"The time has come," the Walrus said,')
s

Sentence('"The time ha... Walrus said,')

In [5]:
for word in s:
    print(word)
    

The
time
has
come
the
Walrus
said


In [6]:
def gen_123():
    yield 1
    yield 2
    yield 3
    
gen_123

<function __main__.gen_123()>

In [7]:
gen_123()

<generator object gen_123 at 0x102a26318>

In [8]:
for i in gen_123():
    print(i)

1
2
3


In [10]:
g = gen_123()
next(g), next(g), next(g)

(1, 2, 3)

In [11]:
def gen_AB():
    print('start')
    yield 'A'
    print('continue')
    yield 'B'
    print('end.')
    
for c in gen_AB():
    print('-->', c)

start
--> A
continue
--> B
end.


## 14.5 Sentence 버전 #4: 느긋한 구현

In [14]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    
    def __init__(self, text):
        self.text = text
        
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    
    def __iter__(self):
        for match in RE_WORD.finditer(self.text):
            yield match.group()
        
s = Sentence('"The time has come," the Walrus said,')
s

Sentence('"The time ha... Walrus said,')

In [15]:
for word in s:
    print(word)

The
time
has
come
the
Walrus
said


## 14.6 Sentence 버전 #5: 제네레이터 표현식

In [16]:
def gen_AB():
    print('start')
    yield 'A'
    print('continue')
    yield 'B'
    print('end.')

res1 = [x*3 for x in gen_AB()]

start
continue
end.


In [17]:
for i in res1:
    print('-->', i)

--> AAA
--> BBB


In [19]:
res2 = (x*3 for x in gen_AB())
res2

<generator object <genexpr> at 0x102a266d8>

In [20]:
for i in res2:
    print('-->', i)

start
--> AAA
continue
--> BBB
end.


In [23]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    
    def __init__(self, text):
        self.text = text
        
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    
    def __iter__(self):
        return (match.group() for match in RE_WORD.finditer(self.text))
        
s = Sentence('"The time has come," the Walrus said,')
s

Sentence('"The time ha... Walrus said,')

In [24]:
for word in s:
    print(word)

The
time
has
come
the
Walrus
said


## 14.8 또 다른 예제: 등차수열 제네레이터

In [37]:
class ArithmeticProgression:
    
    def __init__(self, begin, step, end=None):
        self.begin = begin
        self.step = step
        self.end = end
        
    def __iter__(self):
        result = type(self.begin + self.step)(self.begin)
        forever = self.end is None
        index = 0
        while forever or result < self.end:
            yield result
            index += 1
            result = self.begin + self.step * index

ap = ArithmeticProgression(0, 1, 3)
list(ap)

[0, 1, 2]

In [39]:
ap = ArithmeticProgression(0, .5, 3)
list(ap)

[0.0, 0.5, 1.0, 1.5, 2.0, 2.5]

In [41]:
ap = ArithmeticProgression(0, 1/3, 1)
list(ap)

[0.0, 0.3333333333333333, 0.6666666666666666]

In [43]:
from fractions import Fraction
ap = ArithmeticProgression(0, Fraction(1, 3), 1)
list(ap)

[Fraction(0, 1), Fraction(1, 3), Fraction(2, 3)]

In [45]:
from decimal import Decimal
ap = ArithmeticProgression(0, Decimal('.1'), .3)
list(ap)

[Decimal('0'), Decimal('0.1'), Decimal('0.2')]

In [47]:
def aritprog_gen(begin, step, end=None):
    result = type(begin + step)(begin)
    forever = end is None
    index = 0
    while forever or result < end:
        yield result
        index += 1
        result = begin + step * index
        
ap = ArithmeticProgression(0, 1, 3)
list(ap)
        

[0, 1, 2]

In [1]:
import itertools
gen = itertools.count(1, .5)
next(gen), next(gen), next(gen), next(gen)

(1, 1.5, 2.0, 2.5)

In [3]:
gen = itertools.takewhile(lambda n: n < 3, itertools.count(1, .5))
list(gen)

[1, 1.5, 2.0, 2.5]

In [1]:
import itertools

def aritprog_gen(begin, step, end=None):
    first = type(begin + step)(begin)
    ap_gen = itertools.count(first, step)
    if end is not None:
        ap_gen = itertools.takewhile(lambda n: n < end, ap_gen)
    return ap_gen

list(aritprog_gen(0, 1, 3))

[0, 1, 2]

## 14.9 표준 라이브러리의 제네레이터 함수

In [3]:
def vowel(c):
    return c.lower() in 'aeiou'

list(filter(vowel, 'Aardvark'))

['A', 'a', 'a']

In [4]:
import itertools
list(itertools.filterfalse(vowel, 'Aardvark'))

['r', 'd', 'v', 'r', 'k']

In [5]:
list(itertools.dropwhile(vowel, 'Aardvark'))

['r', 'd', 'v', 'a', 'r', 'k']

In [6]:
list(itertools.takewhile(vowel, 'Aardvark'))

['A', 'a']

In [7]:
list(itertools.compress('Aardvark', (1, 0, 1, 1, 0, 1)))

['A', 'r', 'd', 'a']

In [8]:
list(itertools.islice('Aardvark', 4))

['A', 'a', 'r', 'd']

In [9]:
list(itertools.islice('Aardvark', 4, 7))

['v', 'a', 'r']

In [11]:
list(itertools.islice('Aardvark', 1, 7, 2))

['a', 'd', 'a']

In [2]:
sample = [5, 4, 2, 8, 7, 6, 3, 0, 9, 1]
import itertools
list(itertools.accumulate(sample))

[5, 9, 11, 19, 26, 32, 35, 35, 44, 45]

In [15]:
list(itertools.accumulate(sample, min))

[5, 4, 2, 2, 2, 2, 2, 0, 0, 0]

In [19]:
list(itertools.accumulate(sample, max))

[5, 5, 5, 8, 8, 8, 8, 8, 9, 9]

In [6]:
import operator
list(itertools.accumulate(sample, operator.mul))

[5, 20, 40, 320, 2240, 13440, 40320, 0, 0, 0]

In [23]:
list(itertools.accumulate(range(1, 11), operator.mul))

[1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]

In [1]:
list(enumerate('albatroz', 1))

[(1, 'a'),
 (2, 'l'),
 (3, 'b'),
 (4, 'a'),
 (5, 't'),
 (6, 'r'),
 (7, 'o'),
 (8, 'z')]

In [3]:
import operator
list(map(operator.mul, range(11), range(11)))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100]

In [4]:
list(map(operator.mul, range(11), [2, 4, 8]))

[0, 4, 16]

In [5]:
list(map(lambda a, b: (a, b), range(11), [2, 4, 8]))

[(0, 2), (1, 4), (2, 8)]

In [6]:
import itertools
list(itertools.starmap(operator.mul, enumerate('albatroz', 1)))

['a', 'll', 'bbb', 'aaaa', 'ttttt', 'rrrrrr', 'ooooooo', 'zzzzzzzz']

In [7]:
sample = [5, 4, 2, 8, 7, 6, 3, 0, 9, 1]
list(itertools.starmap(lambda a, b: b/a, enumerate(itertools.accumulate(sample), 1)))

[5.0,
 4.5,
 3.6666666666666665,
 4.75,
 5.2,
 5.333333333333333,
 5.0,
 4.375,
 4.888888888888889,
 4.5]

In [8]:
list(itertools.chain('ABC', range(2)))

['A', 'B', 'C', 0, 1]

In [9]:
list(itertools.chain.from_iterable(enumerate('ABC')))

[0, 'A', 1, 'B', 2, 'C']

In [10]:
list(zip('ABC', range(5)))

[('A', 0), ('B', 1), ('C', 2)]

In [11]:
list(itertools.zip_longest('ABC', range(5)))

[('A', 0), ('B', 1), ('C', 2), (None, 3), (None, 4)]

In [13]:
list(itertools.zip_longest('ABC', range(5), fillvalue='?'))

[('A', 0), ('B', 1), ('C', 2), ('?', 3), ('?', 4)]

In [14]:
list(itertools.product('ABC', range(2)))

[('A', 0), ('A', 1), ('B', 0), ('B', 1), ('C', 0), ('C', 1)]

In [15]:
suits = 'spades hearts diamonds clubs'.split()
list(itertools.product('AK', suits))

[('A', 'spades'),
 ('A', 'hearts'),
 ('A', 'diamonds'),
 ('A', 'clubs'),
 ('K', 'spades'),
 ('K', 'hearts'),
 ('K', 'diamonds'),
 ('K', 'clubs')]

In [16]:
list(itertools.product('ABC'))

[('A',), ('B',), ('C',)]

In [17]:
list(itertools.product('ABC', repeat=2))

[('A', 'A'),
 ('A', 'B'),
 ('A', 'C'),
 ('B', 'A'),
 ('B', 'B'),
 ('B', 'C'),
 ('C', 'A'),
 ('C', 'B'),
 ('C', 'C')]

In [18]:
list(itertools.product(range(2), repeat=3))

[(0, 0, 0),
 (0, 0, 1),
 (0, 1, 0),
 (0, 1, 1),
 (1, 0, 0),
 (1, 0, 1),
 (1, 1, 0),
 (1, 1, 1)]

In [20]:
rows = itertools.product('AB', range(2), repeat=2)
for row in rows: print(row)

('A', 0, 'A', 0)
('A', 0, 'A', 1)
('A', 0, 'B', 0)
('A', 0, 'B', 1)
('A', 1, 'A', 0)
('A', 1, 'A', 1)
('A', 1, 'B', 0)
('A', 1, 'B', 1)
('B', 0, 'A', 0)
('B', 0, 'A', 1)
('B', 0, 'B', 0)
('B', 0, 'B', 1)
('B', 1, 'A', 0)
('B', 1, 'A', 1)
('B', 1, 'B', 0)
('B', 1, 'B', 1)


In [23]:
ct = itertools.count()
next(ct), next(ct), next(ct)

(0, 1, 2)

In [26]:
list(itertools.islice(itertools.count(1, .3), 3))

[1, 1.3, 1.6]

In [27]:
cy = itertools.cycle('ABC')
next(cy)

'A'

In [ ]:
list(itertools.islice(cy, 7))

In [3]:
rp = itertools.repeat(7)
next(rp), next(rp)

(7, 7)

In [4]:
list(itertools.repeat(8, 7))

[8, 8, 8, 8, 8, 8, 8]

In [7]:
list(map(operator.mul, range(11), itertools.repeat(5)))

[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]

In [10]:
list(itertools.combinations('ABC', 2))

[('A', 'B'), ('A', 'C'), ('B', 'C')]

In [12]:
list(itertools.combinations_with_replacement('ABC', 2))

[('A', 'A'), ('A', 'B'), ('A', 'C'), ('B', 'B'), ('B', 'C'), ('C', 'C')]

In [13]:
list(itertools.permutations('ABC', 2))

[('A', 'B'), ('A', 'C'), ('B', 'A'), ('B', 'C'), ('C', 'A'), ('C', 'B')]

In [14]:
list(itertools.product('ABC', repeat=2))

[('A', 'A'),
 ('A', 'B'),
 ('A', 'C'),
 ('B', 'A'),
 ('B', 'B'),
 ('B', 'C'),
 ('C', 'A'),
 ('C', 'B'),
 ('C', 'C')]

In [19]:
list(itertools.groupby('LLLLAAGGG'))

[('L', <itertools._grouper at 0x107c5a860>),
 ('A', <itertools._grouper at 0x107c5ac88>),
 ('G', <itertools._grouper at 0x107c5acf8>)]

In [22]:
for char, group in itertools.groupby('LLLLAAAGG'):
    print(char, '->', list(group))

L -> ['L', 'L', 'L', 'L']
A -> ['A', 'A', 'A']
G -> ['G', 'G']


In [24]:
animals = ['duck', 'eagle', 'rat', 'firaffe', 'bear', 'bat', 'dolphin', 'shark', 'lion']
animals.sort(key=len)
animals

['rat', 'bat', 'duck', 'bear', 'lion', 'eagle', 'shark', 'firaffe', 'dolphin']

In [27]:
for length, group in itertools.groupby(animals, len):
    print(length, '->', list(group))

3 -> ['rat', 'bat']
4 -> ['duck', 'bear', 'lion']
5 -> ['eagle', 'shark']
7 -> ['firaffe', 'dolphin']


In [28]:
for length, group in itertools.groupby(reversed(animals), len):
    print(length, '->', list(group))

7 -> ['dolphin', 'firaffe']
5 -> ['shark', 'eagle']
4 -> ['lion', 'bear', 'duck']
3 -> ['bat', 'rat']


In [31]:
list(itertools.tee('ABC'))

[<itertools._tee at 0x107b9c948>, <itertools._tee at 0x107c61ac8>]

In [36]:
g1, g2 = itertools.tee('ABC')
next(g1)

'A'

In [37]:
next(g2)

'A'

In [38]:
next(g2)

'B'

In [39]:
list(g1)

['B', 'C']

In [40]:
list(g2)

['C']

In [41]:
list(zip(*itertools.tee('ABC')))

[('A', 'A'), ('B', 'B'), ('C', 'C')]

## 14.10 파이썬 3.3의 새로운 구문: yield from

In [43]:
def chain(*iterable):
    for it in iterable:
        for i in it:
            yield i
            
s = 'ABC'
t = tuple(range(3))
list(chain(s, t))

['A', 'B', 'C', 0, 1, 2]

In [44]:
def chain(*iterable):
    for it in iterable:
        yield from it
list(chain(s, t))

['A', 'B', 'C', 0, 1, 2]

## 14.11 반복형을 리듀스하는 함수

In [1]:
all([1, 2, 3])

True

In [2]:
all([1, 0, 3])

False

In [3]:
all([])

True

In [4]:
any([1, 2, 3])

True

In [5]:
any([1, 0, 3])

True

In [7]:
any([0, 0.0])

False

In [6]:
any([])

False

In [8]:
g = (n for n in [0, 0.0, 7, 8])
any(g)

True

In [9]:
next(g)

8

## 14.12 iter() 함수 들여다보기

In [13]:
from random import randint

def d6():
    return randint(1, 6)

d6_iter = iter(d6, 1)
d6_iter

In [14]:
for roll in d6_iter:
    print(roll)

3
2


## 뒷 이야기

In [1]:
def f():
    x=0
    while True:
        x += 1
        yield x
        
gen = f()
next(gen), next(gen), next(gen)

(1, 2, 3)

In [ ]:
def f():
    def do_yield(n):
        yield n
    x = 0
    while True:
        x += 1
        do_yield(x)
        
gen = f()
gen

In [3]:
def f():
    def do_yield(n):
        yield n
    x = 0
    while True:
        x += 1
        yield from do_yield(x)
        
gen = f()
next(gen), next(gen), next(gen)

(1, 2, 3)

In [16]:
from collections import abc
e = enumerate('ABC')
isinstance(e, abc.Iterable)

True

In [17]:
import types
e = enumerate('ABC')
isinstance(e, types.GeneratorType)

False

In [18]:
isinstance(gen, types.GeneratorType)

True

In [24]:
class Fibonacci:
    
    def __iter__(self):
        return FibonacciGeneartor()
    
class FibonacciGeneartor:
    
    def __init__(self):
        self.a = 0
        self.b = 1
        
    def __next__(self):
        result = self.a
        self.a, self.b = self.b, self.a + self.b
        return result
    
    def __iter__(self):
        return self
    
fibonacci = iter(Fibonacci())
for idx in range(10):
    print(next(fibonacci))

0
1
1
2
3
5
8
13
21
34


In [25]:
def fibonacci():
    a, b = 0, 1
    while True:
        yield a
        a, b = b, a + b

fibo = fibonacci()
for idx in range(10):
    print(next(fibo))

0
1
1
2
3
5
8
13
21
34
